In [1]:
# import dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim import models, corpora, similarities
import re
from nltk.stem.porter import PorterStemmer
import time
from nltk import FreqDist
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim
from gensim.matutils import kullback_leibler, jaccard, hellinger
import logging
import pickle
import warnings
warnings.filterwarnings('ignore')

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
print('Downloads Complete')

Downloads Complete


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
def get_most_similar_documents(query, corpus, dictionary, k=10):
    distances = []
    for c in corpus:
        distances.append(kullback_leibler(query, c, num_features=len(dictionary)))
    
    indices = np.array(distances).argsort()[:k]
    return indices

def get_most_similar_businesses(query_data, corpus, dictionary, model):
    query_bow = dictionary.doc2bow(query_data)
    most_sim_ids = get_most_similar_documents(model[query_bow], model[corpus], dictionary)
    return most_sim_ids

def get_dictionary_corpus(data, no_below=5, no_above=0.1):
    dictionary = corpora.Dictionary(data)
    #dictionary.filter_extremes(no_below=no_below, no_above=no_above)
    corpus = [dictionary.doc2bow(doc) for doc in data]
    return dictionary, corpus

In [29]:
def train_lda(corpus, id2word, chunksize=2000, num_topics=10, alpha='auto', eta='auto', passes=1, iterations=50,
              minimum_probability=0.01, eval_every=10, random_state=None):
    """
    This function trains the lda model
    We setup parameters like number of topics, the chunksize to use in Hoffman method
    """
    t1 = time.time()
    # low alpha means each document is only represented by a small number of topics, and vice versa
    # low eta means each topic is only represented by a small number of words, and vice versa
    lda = LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=alpha, eta=eta, 
                   chunksize=chunksize, minimum_probability=minimum_probability, passes=passes, 
                   iterations=iterations, eval_every=eval_every, random_state=random_state)
    
    t2 = time.time()
    print("Time to train LDA model on businesses: ", (t2-t1)/60, "min")
    
    return lda

In [30]:
csv_names = ["pittsburgh_reviews_cleaned.csv", "mesa_reviews_cleaned.csv", "charlotte_reviews_cleaned.csv"]

In [31]:
for csv in csv_names:
    df = pd.read_csv(csv, index_col=0)
    df['tokenized'] = df['tokenized'].apply(eval)
    dictionary, corpus = get_dictionary_corpus(df['tokenized'])
    lda_model = train_lda(corpus=corpus, id2word=dictionary)
    pickle.dump(dictionary, open(csv+"_dictionary.pkl","wb"))
    pickle.dump(corpus,open(csv+"_corpus.pkl","wb"))
    pickle.dump(lda_model,open(csv+"_lda.pkl","wb"))
    print(csv,"done")
    

Time to train LDA model on businesses:  0.16230140527089437 min
pittsburgh_reviews_cleaned.csv done
Time to train LDA model on businesses:  0.07581223646799723 min
mesa_reviews_cleaned.csv done
Time to train LDA model on businesses:  0.16315337419509887 min
charlotte_reviews_cleaned.csv done


In [32]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      13.695041        1       1 -0.004680 -0.000329
2      11.905770        1       2  0.002384  0.006075
4      10.903715        1       3 -0.001426  0.001773
5       9.981760        1       4 -0.001027 -0.002557
6       9.652217        1       5 -0.003066 -0.000526
3       9.593349        1       6 -0.003253  0.001986
9       9.177608        1       7  0.001509 -0.001740
1       8.614253        1       8  0.002557 -0.001954
8       8.338760        1       9  0.002329 -0.003877
0       8.137527        1      10  0.004673  0.001151, topic_info=     Category          Freq        Term         Total  loglift  logprob
2054  Default   5689.000000        thai   5689.000000  30.0000  30.0000
1182  Default   8076.000000         bbq   8076.000000  29.0000  29.0000
1779  Default   9560.000000        wing   9560.000000  28.0000  28.0000
1470  Default   1904.000000       bagel   1904.000000  27.0000  27.0000
2765  Default  10690.000000       manag  10690.000000  26.0000  26.0000
3193  Default  10141.000000        meat  10141.000000  25.0000  25.0000
1848  Default  10140.000000        need  10140.000000  24.0000  24.0000
2827  Default  10503.000000        pork  10503.000000  23.0000  23.0000
882   Default   9941.000000       tasti   9941.000000  22.0000  22.0000
3174  Default   6890.000000         mac   6890.000000  21.0000  21.0000
462   Default   4769.000000       salsa   4769.000000  20.0000  20.0000
237   Default   1673.000000       donut   1673.000000  19.0000  19.0000
504   Default   6594.000000        help   6594.000000  18.0000  18.0000
2319  Default   8892.000000        rice   8892.000000  17.0000  17.0000
1782  Default   7021.000000         ill   7021.000000  16.0000  16.0000
2308  Default   9886.000000        wine   9886.000000  15.0000  15.0000
1035  Default   7032.000000        made   7032.000000  14.0000  14.0000
2339  Default  10732.000000      around  10732.000000  13.0000  13.0000
3246  Default   3353.000000         pho   3353.000000  12.0000  12.0000
4130  Default   2488.000000     airport   2488.000000  11.0000  11.0000
2834  Default  10305.000000      expect  10305.000000  10.0000  10.0000
3570  Default   8124.000000        year   8124.000000   9.0000   9.0000
66    Default   3771.000000       slice   3771.000000   8.0000   8.0000
3598  Default   7694.000000       spici   7694.000000   7.0000   7.0000
153   Default   2551.000000       ramen   2551.000000   6.0000   6.0000
592   Default  11536.000000         see  11536.000000   5.0000   5.0000
2082  Default   4257.000000     mexican   4257.000000   4.0000   4.0000
3383  Default   2543.000000     burrito   2543.000000   3.0000   3.0000
2486  Default   4424.000000       pasta   4424.000000   2.0000   2.0000
3152  Default  10489.000000        park  10489.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
3222  Topic10   1050.749023        mani  10880.293945   0.1712  -5.6673
1782  Topic10    720.521912         ill   7021.496094   0.2319  -6.0446
3743  Topic10   1068.971191      though  11547.480469   0.1289  -5.6501
1847  Topic10    739.714355        fish   7394.523926   0.2065  -6.0183
4517  Topic10   1045.645630         bit  11344.296875   0.1246  -5.6722
1906  Topic10    892.247681      custom   9415.333984   0.1523  -5.8308
1035  Topic10    700.156311        made   7032.476562   0.2017  -6.0733
1036  Topic10    991.785583      select  10861.726562   0.1152  -5.7250
713   Topic10    489.247864        done   4565.013672   0.2754  -6.4317
2319  Topic10    828.027954        rice   8892.502930   0.1348  -5.9055
1491  Topic10    767.343262     qualiti   8092.087891   0.1530  -5.9816
1249  Topic10   1001.551514      shrimp  11529.723633   0.0653  -5.7152
4735  Topic10    939.010925        last  10670.535156   0.0783  -5.7797
3378  Topic10    792.689941      attent   8515.266602   0.1345  -5.94

'accomod'